# Python 类型标注（Typing）教程

## 简介

Python 是一种动态类型语言，这意味着变量的类型是在运行时确定的，而不是在编译时。虽然这提供了灵活性，但也可能导致一些难以追踪的错误。从 Python 3.5 开始，引入了 typing 模块，允许开发者添加类型提示（type hints）来增强代码的可读性和可维护性。

## 基础使用
### 基本类型标注

In [2]:
# 变量类型标注
age: int = 25
name: str = "Alice"
is_student: bool = True
height: float = 1.75

# 函数参数和返回值标注
def greet(name: str) -> str:
    return f"Hello, {name}"

print(greet(name))

Hello, Alice


In [ ]:
# 标注类型不影响运行时的行为
# 这样做不会报错，因为 Python 并不强制执行类型限制
age: int = "twenty five"  # 不符合类型标注，但代码仍会运行

## 常用类型

In [5]:
# 基本类型
from typing import List, Dict, Tuple, Set, Optional, Union, Any

# 列表
names: List[str] = ["1","2","3"]

# 字典
user_ages: Dict[str, int] = {"a":15,"b":20}

# 元组
point: Tuple[int, int] = (10,20)

# 集合
unique_numbers: Set[int] = {1,2,3,4}

# 联合类型
is_number: Union[int, str] = "1234" # 可以是整数或者字符串

# 可选类型
maybe_name: Optional[str] = None # 等价于Union[str, None]

# Any 类型 可以是任何类型，相等于不做检查

anything: Any = 42
anything = "123"

## 嵌套类型

In [7]:
# 嵌套类型
matrix: List[List[int]] = [[1,2],[2,3]]

# 复杂的类型嵌套解构
user_by_depth: Dict[str, List[Dict[str, Union[str, int]]]] = {
    "E": [
        {"name":"A","age":25},
        {"name":"B","age":30}
    ]
}


# 自定义类型与类型别名

In [12]:
from typing import List, Dict, Union, TypeAlias

# 类型别名
UserID = int
UserName= str
userdata = Dict[str, Union[str, int]]

# 使用类型别名
userid: UserID = 12345
userData: Dict[str, Union[str, int]] = {1:"A","2":"b"}

# Python 3.10+ 支持更简洁的语法
UserList: TypeAlias = List[userdata]
UserList = List[userdata]

In [ ]:
## 函数和可调用对象

In [ ]:
from typing import Callable

# Callable 接收两个部分,一个列表 [...]，表示参数类型,一个返回值类型

# 接受任意输入，返回str
# GreetingFunc = Callable[..., str]

# # 不接受任何输入，返回str
# GreetingFunc = Callable[[], str]

# 函数类型，接受一个入参str，一个返回str
GreetingFunc = Callable[[str], str]

def create_greeter(greeting_func: GreetingFunc) -> GreetingFunc:
    def wrapper(name: str) -> str:
        return greeting_func(name)
    return wrapper

def hello(name: str) -> str:
    return f"hello ,{name}!"

new_greeter = create_greeter(hello)

print(new_greeter("abc"))

hello ,abc!


## 泛型
泛型允许创建适用于多种类型的代码

In [16]:
from typing import TypeVar, Generic, List

T = TypeVar('T')

# 泛型函数
def first_element(items: List[T]) -> T:
    return items[0]


# 泛型类
class Stack(Generic[T]):
    def __init__(self) -> None:
        self.items: List[T] = []

    def push(self, item: T) -> None:
        self.items.append(item)
    
    def pop(self) -> T:
        return self.items.pop()
    
    def is_empty(self) -> bool:
        return len(self.items) == 0
    
# 使用例子
int_stack = Stack[int]()
int_stack.push(1)
int_stack.push(2)
print(int_stack.pop())

str_stack = Stack[str]()
str_stack.push('hello')
print(str_stack.pop())

2
hello


## 实际使用案例
### 数据处理

In [18]:
from typing import Dict, List, Tuple, Optional, Union, TypedDict, Callable
import json
from datetime import datetime

# 使用TypeDict定义结构化字典类型
class UserDict(TypedDict):
    id: int
    name: str
    email: str
    active: bool
    joined_at: str
    metadata: Optional[Dict[str, Union[str, int, bool]]]

# 定义处理函数
def load_users(file_path: str) -> List[UserDict]:
    with open(file_path, 'r') as f:
        return json.load(f)
    
def filter_users(
        users: List[UserDict],
        predicate: Callable[[UserDict], bool]
) -> List[UserDict]:
    return [user for user in users if predicate(user)]

def get_user_stars(users: List[UserDict]) -> Dict[str, Union[int, float]]:
    active_users = filter_users(users, lambda u:u["active"])
    return {
        "total": len(users),
        "active": len(active_users),
        "active_percentage": len(active_users)/ len(users) *100 if users else 0
    }

def format_date(date_str: str) -> str:
    dt = datetime.fromisoformat(date_str.replace("Z","+00:00"))
    return dt.strftime("%Y-%m-%d")

def process_users(file_path: str) -> Tuple[List[UserDict], Dict[str, Union[int, float]]]:
    users = load_users(file_path)

    for user in users:
        # 添加格式化日期
        if "joined_at" in user:
            user["formatted_join_date"] = format_date(user["joined_at"])
    
    stats = get_user_stars(users)
    return users, stats

users, stats = process_users("user.json")
print(f"共有 {stats['total']} 个用户，其中 {stats['active']} 个活跃用户")

共有 3 个用户，其中 2 个活跃用户
